In [ ]:
from teehr import Evaluation
from pathlib import Path

In [ ]:
# Set a path to the directory where the evaluation will be created
TEST_STUDY_DIR = Path(Path().home(), "temp", "test_study")
# TEST_STUDY_DIR = Path("/home/sam/git/teehr/tests/data/v0_3_test_study")

# Create an Evaluation object
ev = Evaluation(dir_path=TEST_STUDY_DIR)

# Enable logging
ev.enable_logging()

In [ ]:
ts_df = ev.primary_timeseries.load_parquet().to_pandas()
ts_df.head()

In [ ]:
# Pass in SQL format "where" clauses as chained filter calls.
# Note, no internal validation is done when filtering with SQL formatted strings.
ts_df = ev.primary_timeseries.filter(
    "value_time > '2022-01-01' and value_time < '2022-01-02'"
).filter(
    "location_id = 'gage-C'"
).to_pandas()
ts_df.head()

In [ ]:
# Pass in a list of dictionaries with "column", "operator", and "value" keys
# to chain filters.
ts_df = ev.primary_timeseries.filter(
    [
        {
            "column": "value_time",
            "operator": ">",
            "value": "2022-01-01",
        },
        {
            "column": "value_time",
            "operator": "<",
            "value": "2022-01-02",
        },
        {
            "column": "location_id",
            "operator": "=",
            "value": "gage-C",
        },
    ]
).to_pandas()
ts_df.head()

In [ ]:
# Using the query method to filter the timeseries where filters are passed in as arguments.
ts_df = ev.primary_timeseries.query(
    filters=[
        "value_time > '2022-01-01'",
        "value_time < '2022-01-02'",
        "location_id = 'gage-C'"
    ]
).to_pandas()
ts_df.head()

In [ ]:
ts_df = ev.primary_timeseries.query(
    filters=[
        {
            "column": "value_time",
            "operator": ">",
            "value": "2022-01-01",
        },
        {
            "column": "value_time",
            "operator": "<",
            "value": "2022-01-02",
        },
        {
            "column": "location_id",
            "operator": "=",
            "value": "gage-C",
        },
    ],
    order_by=["location_id", "value_time"]
).to_pandas()
ts_df.head()

In [ ]:
from teehr.models.filters import TimeseriesFilter
from teehr.models.filters import FilterOperators
fields = ev.primary_timeseries.field_enum()

In [ ]:
ts_df = ev.primary_timeseries.query(
    filters=[
        TimeseriesFilter(
            column=fields.value_time,
            operator=FilterOperators.gt,
            value="2022-01-01",
        ),
        TimeseriesFilter(
            column=fields.value_time,
            operator=FilterOperators.lt,
            value="2022-01-02",
        ),
        TimeseriesFilter(
            column=fields.location_id,
            operator=FilterOperators.eq,
            value="gage-C",
        ),
]).to_pandas()
ts_df.head()

In [ ]:
# Using the query method to filter the timeseries where filters are passed in as arguments.
# The result is returned as a Spark DataFrame, which is further "selected" and "ordered" before
# being converted to a pandas DataFrame.
ts_sdf = ev.primary_timeseries.query(
    filters=[
        "value_time > '2022-01-01'",
        "value_time < '2022-01-02'",
        "location_id = 'gage-C'"
    ]
).to_sdf()
ts_df = ts_sdf.select("value_time", "location_id", "value").orderBy("value").toPandas()
ts_df.head()